<center>
    <h1> Programación Científica</h1>
    <h2> Actividad 5 </h2>
    <h2> Felipe Chacón Ossa </h2>
    <h2> 201303017-3 </h2>
</center>

_Mayo 2017_

In [1]:
%matplotlib inline

import numpy as np
import numexpr as ne
import numba
import math
import random
import matplotlib.pyplot as plt
import scipy as sp
import sys

%load_ext Cython

## La distancia de Hausdorff nuevamente...

En esta actividad volveremos a implementar la distancia/métrica de Hausdorff, pero ahora utilizando Cython.

__La métrica de Hausdorff__ corresponde a un métrica o distancia ocupada para medir cuán disímiles son dos subconjuntos dados. 

Esta tiene muchas aplicaciones, en particular para comparar el parecido entre imágenes. En el caso en donde los conjuntos son arreglos bidimensionales, la definición es la siguiente:

Sean $X \in \mathbb{R}^{m \times 3}$ e  $Y \in \mathbb{R}^{n \times 3}$ dos matrices, la métrica/distancia de Hausdorff sobre sobre estas como:

$$
d_H(X,Y) = \max \left(\ \max_{i\leq m} \min_{j \leq n} d(X[i],Y[j]), \ \max_{j\leq n} \min_{i \leq m} d(Y[j],X[i]) \ \right)
$$

donde $d$ es la _distancia Euclideana_ clásica. ($X[i]$ indíca la i-ésima fila de X).

__Ilustración unidimensional:__ Distancia entre funciones.
<img src='data/hausdorff.png' style="width: 600px;">

A continuación se le proveen 3 funciones que implementan tal métrica, usando __Numba__.

In [2]:
@numba.jit('float64 (float64[:], float64[:])')
def metric_numba(x, y):
    """
    standard Euclidean distance
    """
    ret = x-y
    ret *= ret
    return np.sqrt(ret).sum()


@numba.jit('float64 (float64[:], float64[:,:])', nopython=True)
def inf_dist_numba(x, Y):
    """
    inf distance between row x and array Y
    """
    m = Y.shape[0]
    inf = np.inf
    
    for i in range(m):
        dist = metric_numba(x, Y[i])
        if dist < inf:
            inf = dist
    return inf

@numba.jit('float64 (float64[:,:], float64[:,:])', nopython=True)
def hausdorff_numba(X, Y):
    """
    Hausdorff distance between arrays X and Y
    """
    m = X.shape[0]
    n = Y.shape[0]
    sup1 = -1.
    sup2 = -1.
    
    for i in range(m):
        inf1 = inf_dist_numba(X[i], Y)
        if inf1 > sup1:
            sup1 = inf1
    for i in range(n):
        inf2 = inf_dist_numba(Y[i], X)
        if inf2 > sup2:
            sup2 = inf2
            
    return max(sup1, sup2)

Se solicita que realice lo siguiente:

1. Escribir el equivalente __Cython__ de las tres funciones anteriores, ocupando todas las optimizaciones posibles: __Compiler directives__, __Memory Views__, __Inline Functions__, __Pure C functions__ o cualquier otra optimización que usted considere conveniente.
2. Cree `10` arreglos $X,Y$ aleatorios, con cantidad creciente de filas, y realice análsis de tiempos de ejecuciones de las versiones __Numba__ y __Cython__ de las funciontes anteriores sobre estos arreglos.
3. Concluya.

In [3]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

import numpy as np
cimport numpy as cnp
ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

cdef inline float64_t metric_cython(float64_t[::1] x, float64_t[::1] y):
    """
    standard Euclidean distance
    """
    cdef:
        int i = 0
        int n = x.shape[0]
        float ret = 0
    for i in range(n):
        ret += (x[i]-y[i])**2
    return sqrt(ret)

cdef float64_t inf_dist_cython(float64_t[::1] x, float64_t[:,::1] Y):
    """
    inf distance between row x and array Y
    """
    cdef int m = Y.shape[0]
    inf = np.inf
    
    for i in range(m):
        dist = metric_cython(x, Y[i])
        if dist < inf:
            inf = dist
    return inf

cpdef float64_t hausdorff_cython(float64_t[:,::1] X, float64_t[:,::1] Y):
    """
    Hausdorff distance between arrays X and Y
    """
    cdef:
        int m = X.shape[0]
        int n = Y.shape[0]
        float sup1 = -1.
        float sup2 = -1.
    
    for i in range(m):
        inf1 = inf_dist_cython(X[i], Y)
        if inf1 > sup1:
            sup1 = inf1
    for i in range(n):
        inf2 = inf_dist_cython(Y[i], X)
        if inf2 > sup2:
            sup2 = inf2
            
    return max(sup1, sup2)

## Crear 10 arreglos X y 10 Y aleatorias

In [4]:
nrows = np.linspace(100,5000,10).astype(int)

for nrow in nrows:
    X = np.random.random((nrow,3))
    Y = np.random.random((nrow,3))
    tc = %timeit -o hausdorff_cython(X,Y)
    tn = %timeit -o hausdorff_numba(X,Y)
    print("Number of rows: {0}".format(nrow))
    print("Best time in Cython: {0}".format(tc.best))
    print("Best time in Numba: {0} \n".format(tn.best))
    del X,Y

1000 loops, best of 3: 821 µs per loop
100 loops, best of 3: 6.68 ms per loop
Number of rows: 100
Best time in Cython: 0.0008209420849998423
Best time in Numba: 0.006681909719991381 

10 loops, best of 3: 32.5 ms per loop
1 loop, best of 3: 270 ms per loop
Number of rows: 644
Best time in Cython: 0.03249728629998572
Best time in Numba: 0.2698059940012172 

10 loops, best of 3: 114 ms per loop
1 loop, best of 3: 916 ms per loop
Number of rows: 1188
Best time in Cython: 0.11385776309980429
Best time in Numba: 0.9163219410002057 

1 loop, best of 3: 247 ms per loop
1 loop, best of 3: 1.94 s per loop
Number of rows: 1733
Best time in Cython: 0.24713123699984862
Best time in Numba: 1.9443926999992982 

1 loop, best of 3: 419 ms per loop
1 loop, best of 3: 3.34 s per loop
Number of rows: 2277
Best time in Cython: 0.41901243600295857
Best time in Numba: 3.3403247839996766 

1 loop, best of 3: 626 ms per loop
1 loop, best of 3: 5.1 s per loop
Number of rows: 2822
Best time in Cython: 0.6261369

## Conclusión
Si bien se apreciaba una optimización al hacer el paso a paso del traspaso a Cython, es decir, definir los tipos, usar las directivas, memory view, etc. La mayor optimización, la cual superó el tiempo de Numba, se observó al escribir como Pure Function la función metric_cython, esto debido a que es la función que se llama más veces, por lo que una mejora en el tiempo de esta función se magnifica hasta mostrar resultados bastante notorios.
Con todas las optimizaciones, el código Cython es mejor en aproximadamente 1 orden de magnitud al código Numba.